In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("raw_data/ddp_benchmark.csv")
df = df.rename(columns={"batch_size": "Batch"})

In [ ]:
STEP_COL = "Step Time ($\\mu \\pm \\sigma$)"
COMM_COL = "Comm Time ($\\mu \\pm \\sigma$)"

def combine(dff, new_col, mean_col, std_col):
    dff[new_col] = dff[mean_col].round(2).astype(str) + " ± " + dff[std_col].round(2).astype(str) + " ms"
    
def process_sync_type(sync_type, opt_sharding=False):
    df1 = df[(df.sync_type==sync_type) & (df.sharded_optimizer == opt_sharding)].copy()
    combine(df1, STEP_COL, "step_time_mean", "step_time_std")
    combine(df1, COMM_COL, "comm_time_mean", "comm_time_std")
    df1["Comm Frac"] = (df1["comm_time_frac"] * 100).round(2).astype(str) + "\\%"
    df1 = df1.rename(columns={"batch_size": "Batch"})[["Batch", STEP_COL, COMM_COL, "Comm Frac"]]
    return df1

In [ ]:
def create_latex(report, caption):
    tex = report.to_latex(index=False, caption=caption, escape=False)
    tex = tex.replace(r"\begin{table}", r"\begin{table}[H]\centering")
    print(tex)

In [ ]:
naive = process_sync_type("individual")
naive

In [ ]:
create_latex(naive, "Naïve DDP")

In [ ]:
batch = process_sync_type("batch")
batch

In [ ]:
create_latex(batch, "Flat DDP")

In [ ]:
IND_STEP_COL = "Ind Overlap Step Time"
NAIVE_STEP_COL = "Naïve Step Time"
NAIVE_COMM_COL = "Naïve Comm Time"
COMM_IMPROV = "Comm Improvement"

naive2 = df.loc[df.sync_type=="individual", ["Batch", "step_time_mean", "comm_time_mean"]].rename(columns={"step_time_mean": NAIVE_STEP_COL, "comm_time_mean": NAIVE_COMM_COL})
ind = df.loc[(df.sync_type=="overlap_individual") & ~df.sharded_optimizer, ["Batch", "step_time_mean"]].rename(columns={"step_time_mean": IND_STEP_COL})
individual = ind.merge(naive2)
individual["Comm Improvement"] = ((individual[NAIVE_COMM_COL] - (individual[IND_STEP_COL] - (individual[NAIVE_STEP_COL] - individual[NAIVE_COMM_COL]))) / individual[NAIVE_COMM_COL] * 100).round(1).astype(str) + "\\%"
for col in (IND_STEP_COL, NAIVE_STEP_COL, NAIVE_COMM_COL):
    individual[col] = individual[col].round(1).astype(str) + " ms"
individual

In [ ]:
create_latex(individual, "Overlap Individual Parameters")

In [ ]:
def get_name(x):
    return f"{x} MB Bucket"
    
bucket = df.loc[(~df.sharded_optimizer) & df.bucket_size_mb.notnull(), ["Batch", "bucket_size_mb", "step_time_mean"]]
bucket.bucket_size_mb = bucket.bucket_size_mb.astype(int)
bucket = (
    bucket.pivot(index="Batch", columns="bucket_size_mb", values="step_time_mean")
      .rename(columns=get_name)
      .reset_index()
)
for x in (1, 10, 100, 1000):
    col = get_name(x)
    bucket[col] = bucket[col].round(1).astype(str) + " ms"
bucket

In [ ]:
ind2 = individual[["Batch", NAIVE_STEP_COL, IND_STEP_COL]].rename(columns={NAIVE_STEP_COL: "Naïve", IND_STEP_COL: "Ind Overlap"})
ind2

In [ ]:
bucket_comp = ind2.merge(bucket)
bucket_comp

In [ ]:
create_latex(bucket_comp, "Overlap Bucketed Parameters")

In [ ]:
ind3 = df.loc[(df.sync_type=="overlap_individual"), ["Batch", "step_time_mean", "sharded_optimizer"]]
ind_not_sharded_opt = ind3[~ind3.sharded_optimizer].rename(columns={"step_time_mean": "Not Sharded Opt"}).drop(columns="sharded_optimizer")
ind_sharded_opt = ind3[ind3.sharded_optimizer].rename(columns={"step_time_mean": "Sharded Opt"}).drop(columns="sharded_optimizer")
comb_sharded = ind_not_sharded_opt.merge(ind_sharded_opt)
for col in ("Not Sharded Opt", "Sharded Opt"):
    comb_sharded[col] = comb_sharded[col].round(1).astype(str) + " ms"
comb_sharded

In [ ]:
create_latex(comb_sharded, "Optimizer Sharding")

In [ ]:
ind_mem = df.loc[(df.sync_type=="overlap_individual"), ["Batch", "mem_peak_init", "mem_peak_before_optim", "mem_peak_after_optim", "sharded_optimizer"]]
ind_mem = ind_mem.rename(columns={"mem_peak_init": "After Init", "mem_peak_before_optim": "Before Optim", "mem_peak_after_optim": "After Optim"})
for col in ("After Init", "Before Optim", "After Optim"):
    ind_mem[col] /= 1024
t1 = ind_mem[~ind_mem.sharded_optimizer].drop(columns="sharded_optimizer").set_index("Batch")
t2 = ind_mem[ind3.sharded_optimizer].drop(columns="sharded_optimizer").set_index("Batch")
fmt = lambda df: df.applymap(lambda v: f"{v:.1f}")

combined = fmt(t1) + " / " + fmt(t2) + " GB"
combined = combined.reset_index()
combined

In [ ]:
create_latex(combined, "Optimizer Sharding - Peak Memory")